In [0]:
%pip install transformers==4.30.2 "unstructured[pdf,docx]==0.10.30" langchain==0.1.5 llama-index==0.9.3 --quiet
%pip install databricks-vectorsearch==0.22 pydantic==1.10.9 mlflow==2.10.1 --quiet
dbutils.library.restartPython()


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Using cached python_poppler-0.4.1.tar.gz (138 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + meson setup /tmp/pip-install-m23f0dtm/python-poppler_412c9daf10bc42a0801b1a57470ce8d4 /tmp/pip-install-m23f0dtm/python-poppler_412c9daf10bc42a0801b1a57470ce8d4/.mesonpy-7adz35hj -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt

---------------------------------------------------------------------------
CalledProcessError                        Traceback (most recent call last)
File <command-899513014180833>, line 1
----> 1 get_ipython().run_line_magic('pip', 'install python-poppler')

File /databricks/python/lib/python3.10/site-packages/IPython/core/interactiveshell.py:2417, in InteractiveShell.run_line_magic(self, magic_name, line, _stack_depth)
   2415     kwargs['local_ns'] = self.get_local_scope(stack_depth)
   2416 with self.builtin_trap:
-> 2417     result = fn(*args, **kwargs)
   2419 # The code below prevents the output from being displayed
   2420 # when using magics with decodator @output_can_be_silenced
   2421 # when the last Python token in the expression is a ';'.
   2422 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/dbruntime/PipMagicOverrides.py:33, in PipMagicOverrides.pip(self, line)
     31 @line_magic
     32 def pip(self, line):
---> 33     self

In [0]:
%sh rm -rf /var/cache/apt/archives/* /var/lib/apt/lists/* && sudo apt-get clean && sudo apt-get update && sudo apt-get install poppler-utils tesseract-ocr -y

rm: cannot remove '/var/cache/apt/archives/lock': Permission denied
rm: cannot remove '/var/cache/apt/archives/partial': Permission denied
rm: cannot remove '/var/lib/apt/lists/auxfiles': Permission denied
rm: cannot remove '/var/lib/apt/lists/partial': Permission denied


In [0]:
filename_homereport=f"/Volumes/home-inspection-gpt/default/inspection-report/report1.pdf";
# %sh sudo rm -rf /var/cache/apt/archives/* /var/lib/apt/lists/* && sudo apt-get clean && sudo apt-get update && sudo apt-get install poppler-utils tesseract-ocr -y
from unstructured.partition.auto import partition
from unstructured.partition.pdf import partition_pdf
from pyspark.sql.types import StructType, StructField, StringType
import glob
import re
import requests
import io
import json

In [0]:
%sh sudo rm -rf /var/cache/apt/archives/* /var/lib/apt/lists/* && sudo apt-get clean && sudo apt-get update && sudo apt-get install poppler-utils tesseract-ocr -y

sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


In [0]:
# Remove after import from helpers.py
import os
import re
def extract_text_and_language(elements):

    output = []
    for t in elements:
        element_data = {}
        element_data = {
            "type": str(type(t)),
            "element_id": t.id,
            "text_data": t.text,
        }

        if t.metadata.languages:
            element_data["languages"] = t.metadata.languages[0]
        else:
            element_data["languages"] = None

        if "image" in str(type(t)):
            element_data["image_path"] = t.to_dict()["metadata"]["image_path"]
        else:
            element_data["image_path"] = None

        if ("NarrativeText" in str(type(t))) and (
            "parent_id" in t.to_dict()["metadata"]
        ):
            element_data["parent_id"] = t.to_dict()["metadata"]["parent_id"]
        else:
            element_data["parent_id"] = None

        if ("Table" in str(type(t))) and t.metadata.text_as_html:
            element_data["table_data"] = str(t.metadata.text_as_html)
        else:
            element_data["table_data"] = None

        output.append(element_data)

    return output



def sanitize_filename(filename):
    # Replace all non-alphanumeric characters with underscores and convert to lowercase
    sanitized_filename = re.sub(r'[^a-zA-Z0-9]', '_', filename).lower()
    return sanitized_filename

In [0]:
#  f"/Volumes/home-inspection-gpt/default/inspection-report/report1.pdf",
print (filename_homereport)

pdf_elements_digital = partition_pdf(
    filename=filename_homereport,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    strategy="hi_res",
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=f"/Volumes/home-inspection-gpt/default/pdf-images",
    extract_image_block_output_dir="/Volumes/home-inspection-gpt/default/output-image-block",
)

/Volumes/home-inspection-gpt/default/inspection-report/report1.pdf


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-53ab9f13-a598-4dc8-967d-8cd64e7ba759/lib/python3.10/site-packages/pdf2image/pdf2image.py:581, in pdfinfo_from_path(pdf_path, userpw, ownerpw, poppler_path, rawdates, timeout, first_page, last_page)
    580     env["LD_LIBRARY_PATH"] = poppler_path + ":" + env.get("LD_LIBRARY_PATH", "")
--> 581 proc = Popen(command, env=env, stdout=PIPE, stderr=PIPE)
    583 try:

File /usr/lib/python3.10/subprocess.py:971, in Popen.__init__(self, args, bufsize, executable, stdin, stdout, stderr, preexec_fn, close_fds, shell, cwd, env, universal_newlines, startupinfo, creationflags, restore_signals, start_new_session, pass_fds, user, group, extra_groups, encoding, errors, text, umask, pipesize)
    968             self.stderr = io.TextIOWrapper(self.stderr,
    969                     encoding=encoding, err